In [1]:
#!pip install xgboost
from xgboost import XGBClassifier

import pystac
import pystac_client
import planetary_computer as pc
import requests
import rich.table

from IPython.display import Image


# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
from odc.stac import stac_load
tqdm.pandas()

In [2]:
catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace,
    )
crop_presence_data = pd.read_csv("Crop_Location_Data_20221201.csv")
crop_presence_data.head()

,Latitude and Longitude,Class of Land
0,"(10.323727047081501, 105.2516346045924)",Rice
1,"(10.322364360592521, 105.27843410554115)",Rice
2,"(10.321455902933202, 105.25254306225168)",Rice
3,"(10.324181275911162, 105.25118037576274)",Rice
4,"(10.324635504740822, 105.27389181724476)",Rice


In [3]:
def get_vv_vh(corr, time):
    latlong = corr
    latlong = latlong.replace('(','').replace(')','').replace(' ','').split(',')
    latlong = [float(latlong[0]), float(latlong[1])]
    
    box_size_deg = 0.0005
    min_lon = latlong[1]-box_size_deg/2
    min_lat = latlong[0]-box_size_deg/2
    max_lon = latlong[1]+box_size_deg/2
    max_lat = latlong[0]+box_size_deg/2
    bbox = (min_lon, min_lat, max_lon, max_lat)
    
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time)
    items = list(search.get_all_items())
    resolution = 10  # meters per pixel 
    scale = resolution / 111320.0 # degrees per pixel for crs=4326 
    data = stac_load(items, bands=["vv", "vh"], patch_url=pc.sign, bbox=bbox, crs="EPSG:4326", resolution=scale).isel(time=0)
    
    mean = data.median(dim=['latitude','longitude']).compute()
    return float(mean['vh'].mean()), float(mean['vv'].mean())

In [130]:
def get_hh_hv(corr, time):
    latlong = corr
    latlong = latlong.replace('(','').replace(')','').replace(' ','').split(',')
    latlong = [float(latlong[0]), float(latlong[1])]
    
    box_size_deg = 0.0005
    min_lon = latlong[1]-box_size_deg/2
    min_lat = latlong[0]-box_size_deg/2
    max_lon = latlong[1]+box_size_deg/2
    max_lat = latlong[0]+box_size_deg/2
    bbox = (min_lon, min_lat, max_lon, max_lat)
    
    search = catalog.search(collections=["sentinel-1-rtc"], bbox=bbox, datetime=time)
    items = list(search.get_all_items())
    resolution = 10  # meters per pixel 
    scale = resolution / 111320.0 # degrees per pixel for crs=4326 
    data = stac_load(items, bands=["hh", "hv"], patch_url=pc.sign, bbox=bbox, crs="EPSG:4326", resolution=scale).isel(time=0)
    
    mean = data.median(dim=['latitude','longitude']).compute()
    return float(mean['hv'].mean()), float(mean['hh'].mean())

In [6]:
to_strat_X = crop_presence_data.drop('Class of Land', axis = 1)
to_strat_y = crop_presence_data['Class of Land']
_, X_strat, _, y_strat = train_test_split(to_strat_X, to_strat_y, test_size=0.3, stratify=to_strat_y)

In [7]:
df_strat = pd.concat([X_strat, y_strat], axis = 1)
df_strat = df_strat.reset_index()

In [11]:
time_slice = "2022-01-15/2022-05-31"
vh_vv = []

for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    vh, vv = get_vv_vh(coordinates, time_slice)
    #print(vh, vv)
    vh_vv.append([vh, vv])
vh_vv_data = pd.DataFrame(vh_vv,columns =['vh','vv'])

100%|██████████| 600/600 [43:50<00:00,  4.38s/it]


In [ ]:
time_slice = "2022-01-15/2022-05-31"
hh_hv = []

for coordinates in tqdm(crop_presence_data['Latitude and Longitude']):
    hh, hv = get_vv_vh(coordinates, time_slice)
    print(hh, hv)
    hh_hv.append([hh, hv])
hh_hv_data = pd.DataFrame(vh_vv,columns =['hh','hv'])

In [12]:
VV = np.array(vh_vv_data['vv'])
VH = np.array(vh_vv_data['vh'])
rvi = np.sqrt(1- VV / (VV+VH)) * 4 * (VH / (VV + VH))
rvi = pd.DataFrame({'RVI': rvi})

In [16]:
crop_data = pd.concat([crop_presence_data, vh_vv_data, rvi], axis = 1)
crop_data = crop_data[['vh', 'vv', 'RVI', 'Class of Land']]
crop_data

,vh,vv,RVI,Class of Land
0,0.022579,0.110633,0.279131,Rice
1,0.026226,0.142474,0.245174,Rice
2,0.019677,0.188440,0.116291,Rice
3,0.029277,0.162181,0.239187,Rice
4,0.021837,0.095853,0.319703,Rice
...,...,...,...,...
595,0.068065,0.271413,0.359108,Non Rice
596,0.077354,0.265505,0.428659,Non Rice
597,0.057026,0.262198,0.302010,Non Rice
598,0.051407,0.279444,0.244988,Non Rice


In [92]:
X = crop_data.drop(columns=['Class of Land']).values
y = crop_data['Class of Land'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [93]:
sc = StandardScaler()
rob = RobustScaler()
X_train = rob.fit_transform(X_train)
X_test = rob.transform(X_test)

In [102]:
xgb_cl = XGBClassifier(learning_rate=0.02, gamma = 0.1, reg_alpha=0, reg_lambda=1, subsample = 0.8)
y_train01 = np.where(y_train == 'Rice', 1, 0)
y_test01 = np.where(y_test == 'Rice', 1, 0)
xgb_cl.fit(X_train, y_train01)
preds = xgb_cl.predict(X_train)
acc = accuracy_score(y_train01, preds)
print(acc)
preds = xgb_cl.predict(X_test)
accuracy_score(y_test01, preds)

0.9666666666666667


0.9166666666666666

In [107]:
y_01 = np.where(y == 'Rice', 1, 0)

In [124]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth": [3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05],
    "gamma": [0, 0.25, 1],
    "reg_lambda": [0, 1, 10],
    "scale_pos_weight": [1, 3, 5],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

# Init classifier
xgb_cl = XGBClassifier(objective="binary:logistic")

# Init Grid Search
grid_cv = GridSearchCV(xgb_cl, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")

# Fit
_ = grid_cv.fit(X, y_01)
print(grid_cv.best_score_)
grid_cv.best_params_

0.9574166666666667


{'colsample_bytree': 0.5,
 'gamma': 1,
 'learning_rate': 0.05,
 'max_depth': 4,
 'reg_lambda': 0,
 'scale_pos_weight': 1,
 'subsample': 0.8}

In [125]:
final_cl = XGBClassifier(
    **grid_cv.best_params_,
    objective="binary:logistic"
)
final_cl.fit(X_train, y_train01)
preds_train = final_cl.predict(X_train)
acc = accuracy_score(y_train01, preds_train)
print(acc)
preds_test = final_cl.predict(X_test)
accuracy_score(y_test01, preds_test)

0.9708333333333333


0.9083333333333333

In [31]:
test_file = pd.read_csv('challenge_1_submission_template_correct_columns_fixed.csv')
test_file.head()

time_slice = "2022-01-15/2022-05-15"
sub_vh_vv = []

for coordinates in tqdm(test_file['id']):
    vh, vv = get_vv_vh(coordinates, time_slice)
    #print(vh, vv)
    sub_vh_vv.append([vh, vv])
sub_vh_vv_data = pd.DataFrame(sub_vh_vv,columns =['vh','vv'])

100%|██████████| 250/250 [17:00<00:00,  4.08s/it]


In [66]:
VV = np.array(sub_vh_vv_data['vv'])
VH = np.array(sub_vh_vv_data['vh'])
rvi_sub = np.sqrt(1- VV / (VV+VH)) * 4 * (VH / (VV + VH))
rvi_sub = pd.DataFrame({'RVI': rvi_sub})

In [68]:
df_submision = pd.concat([test_file, sub_vh_vv_data, rvi_sub], axis = 1)
df_submision = df_submision[['vh', 'vv', 'RVI']]
df_submision

,vh,vv,RVI
0,0.023219,0.128806,0.238760
1,0.019368,0.043169,0.689417
2,0.014625,0.060399,0.344264
3,0.003192,0.012423,0.369629
4,0.011087,0.059098,0.251152
...,...,...,...
245,0.005160,0.011760,0.673635
246,0.003683,0.014184,0.374384
247,0.004495,0.010280,0.671207
248,0.018936,0.058669,0.482135


In [69]:
transformed_submission_data = rob.transform(np.array(df_submision))

In [127]:
#### PREDITION OF SUBMISON DATA

final_predictions = final_cl.predict(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)
final_prediction_series

0      1
1      1
2      1
3      0
4      1
      ..
245    0
246    0
247    0
248    1
249    1
Length: 250, dtype: int64

In [128]:
target = np.where(final_prediction_series == 1, 'Rice', 'Non Rice')
submission_df = pd.DataFrame({'id':test_file['id'].values, 'target':target})

In [129]:
submission_df.to_csv('SubXGB2.csv', index = False)